# Generalne importy modułów oraz podstawowa konfiguracja

In [120]:
import yaml
import opendatasets as od
import logging as log
import psycopg2 as ps
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from dataclasses import dataclass
from abc import ABC,abstractclassmethod
from typing import List


log.getLogger().setLevel(log.INFO)

CONFIG_FILE = "config.yaml"

import plotly.io as pio
pio.renderers.default = 'notebook_connected'

# Pobranie danych oraz konfiguracja bazy danych

In [5]:
def download_kaggle_dataset(dataset_url: str, destination_path: str = "."):
    """Function for downloading the kaggle dataset based on a given url."""
    od.download(dataset_url, data_dir=destination_path)

In [6]:
download_kaggle_dataset("https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading 120-years-of-olympic-history-athletes-and-results.zip to C:\Users\MSI\Desktop\120-years-of-olympic-history-athletes-and-results


100%|██████████| 5.43M/5.43M [00:00<00:00, 17.0MB/s]


# Kod do połączenia z bazą danych oraz analizy

In [121]:
class OlympicDatabaseConnectionFactory(object):
    COLUMNS = {
        "NOC_REGIONS": ["NOC", "regions", "notes"],
        "ATHLETE_EVENTS": [
            "ID",
            "Name",
            "Sex",
            "Age",
            "Height",
            "Weight",
            "Team",
            "NOC",
            "Games",
            "Year",
            "Season",
            "City",
            "Sport",
            "Event",
            "Medal",
        ],
    }

    def __init__(self, yaml_file):
        self.config = self._build_config(yaml_file)
        self.connect()

    def _build_config(self, yaml_file):
        log.info("Building the configuration from a provided .yaml file..")
        with open(yaml_file, "r") as f:
            config = yaml.load(f, Loader=yaml.Loader)
        self._check_config(config)
        return config

    def _check_config(self, config):
        log.info("Checking the configuration from a provided .yaml file..")
        assert isinstance(config, dict) == True
        assert set(["dbname", "user", "host", "password", "port"]) == set(
            list(config.keys())
        )
        log.info("The configuration from a provided .yaml file is correct!")

    def _check_connection(self):
        log.info("Checking the database connection..")
        with self.connection:
            with self.connection.cursor() as curs:
                curs.execute("SELECT 123")
                assert curs.fetchall()[0][0] == 123
        log.info("The database connection is correct and established!")

    def connect(self):
        log.info(
            "Establishing the connection to the {} database..".format(
                self.config["dbname"]
            )
        )
        try:
            self.connection = ps.connect(**self.config)
        except Exception as e:
            log.error(
                "There was a problem connecting to the database.. The error is {}".format(
                    e
                )
            )
        self._check_connection()

    def _transform_data(self,output_columns):
        if isinstance(output_columns,str):
            try:
                columns = self.COLUMNS[self.table_name]
            except Exception as e:
                log.error(
                    "There was a problem transforming the data.. The error is {}. Make sure you either supply a new table name to the COLUMNS constant or use the two tables provided.".format(
                        e
                    )
                )
        else:
            columns = output_columns
        self.data = pd.DataFrame(self.data, columns=columns)
        

    def query(self, query,output_columns = "ATHLETE_EVENTS"):
        log.info("Executing a query: {}".format(query))
        with self.connection:
            with self.connection.cursor() as curs:
                try:
                    self.table_name = [x.upper() for x in query.split(" ")][
                        [x.upper() for x in query.split(" ")].index("FROM") + 1
                    ]
                    curs.execute(query)
                    self.data = curs.fetchall()
                    log.info(
                        "Query executed successfully. The number of returned rows is {}".format(
                            len(self.data)
                        )
                    )
                    self._transform_data(output_columns)
                    return self.data

                except Exception as e:
                    log.error(
                        "There was a problem executing the query.. The error is {}".format(
                            e
                        )
                    )
                    return None


In [122]:
@dataclass
class BaseConfig:
    plot_type: str
    plot_config: list
    layout_config: dict

@dataclass
class SubPlotsConfig:
    plot_type: str
    plot_config: dict
    subplots_config: List[BaseConfig]
    layout_config: dict

In [123]:
class PlottingFactory(object):
    def __init__(self,config):
        self.config = config

    def build_plotter(self):
        if self.config.plot_type == "pie":
            log.info(
                "Building the PiePlotter based on {}".format(self.config.plot_type)
            )
            return PiePlotter(self.config)
        elif self.config.plot_type == "subplot":
            log.info(
                "Building the SubPlotter based on {} with given plots: {}".format(self.config.plot_type,",".join([x.plot_type for x in self.config.subplots_config]))
            )
            return SubPlotter(self.config)
        elif self.config.plot_type == "bar":
            log.info(
                "Building the BarPlotter based on {}".format(self.config.plot_type)
            )
            return BarPlotter(self.config)
        elif self.config.plot_type == "line":
            log.info(
                "Building the LinePlotter based on {}".format(self.config.plot_type)
            )
            return LinePlotter(self.config)
        else:
            raise Exception("The plot type {} is not recognized..".format(self.config.plot_type))


In [124]:
class BasePlotter(object):
    def __init__(self, config: BaseConfig):
        log.info("Configuring the plot parameters..")
        self.config = config

    def _check_config(self):
        assert isinstance(self.config, BaseConfig)

    @abstractclassmethod
    def _plot(self):
        pass

    @abstractclassmethod
    def _get_figure(self):
        pass

    @abstractclassmethod
    def _add_traces(self):
        pass

    def plot(self):
        log.info("Displaying the plot..")
        self._plot()
        log.info("Plot rendered successfully!")


In [125]:
class BarPlotter(BasePlotter):
    def _get_figure(self):
        self.fig = go.Figure()
    
    def _add_traces(self, subplot = False):
        self.mode = subplot
        self.traces = [go.Bar(**trace_info) for trace_info in self.config.plot_config]
        for trace in self.traces:
            self.fig.add_trace(trace)
        return self.traces if self.mode else None
    
    def _plot(self):
        self._get_figure()
        self._add_traces()
        self.fig.update_layout(**self.config.layout_config)
        self.fig.show() if self.mode != True else None

class PiePlotter(BasePlotter):
    def _get_figure(self):
        self.fig = go.Figure()
    
    def _add_traces(self):
        for trace_info in self.config.plot_config:
            self.fig.add_trace(go.Pie(**trace_info))
    
    def _plot(self):
        self._get_figure()
        self._add_traces()
        self.fig.update_layout(**self.config.layout_config)
        self.fig.show()

class LinePlotter(BasePlotter):
    def _get_figure(self):
        self.fig = go.Figure()
    
    def _add_traces(self, subplot = True):
        self.mode = subplot
        self.traces = [go.Line(**trace_info) for trace_info in self.config.plot_config]
        for trace in self.traces:
            self.fig.add_trace(trace)
        return self.traces if self.mode else None
    
    def _plot(self):
        self._get_figure()
        self._add_traces()
        self.fig.update_layout(**self.config.layout_config)
        self.fig.show() if self.mode != True else None

class SubPlotter(BasePlotter):
    def _get_figure(self):
        self.fig = make_subplots(**self.config.plot_config)
    
    def _build_plotters(self):
        self.plotters = [build_plot(config) for config in self.config.subplots_config]
    
    def _add_traces(self):
        self.traces = [plotter._add_traces(subplot = True) for plotter in self.plotters]
        self.fig.add_traces(self.traces[0],rows=1,cols=1)
        self.fig.add_traces(self.traces[1],rows=2,cols=1)
    
    def _plot(self):
        self._get_figure()
        self._build_plotters()
        self._add_traces()
        self.fig.update_layout(**self.config.layout_config)
        self.fig.show()
    


In [126]:
def build_plot(config):
    plotter = PlottingFactory(config).build_plotter()
    plotter.plot() 
    return plotter

# Połączenie z bazą danych

In [127]:
connection = OlympicDatabaseConnectionFactory(CONFIG_FILE)

INFO:root:Building the configuration from a provided .yaml file..
INFO:root:Checking the configuration from a provided .yaml file..
INFO:root:The configuration from a provided .yaml file is correct!
INFO:root:Establishing the connection to the postgres database..
INFO:root:Checking the database connection..
INFO:root:The database connection is correct and established!


# Eksploracyjna analiza danych oraz raport

## Zawodnicy z największa ilościa odbytych igrzysk olimpijskich

In [128]:
query = """
WITH grouped_query as (SELECT Name,count(*) 
                       from athlete_events 
                       group by Name)
SELECT * FROM grouped_query order by count desc limit 10;
"""

In [129]:
df = connection.query(query,
    output_columns=["Name","Number of olympics appearances"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Name,count(*) 
                       from athlete_events 
                       group by Name)
SELECT * FROM grouped_query order by count desc limit 10;

INFO:root:Query executed successfully. The number of returned rows is 10


In [130]:
config = BaseConfig(
    plot_type="bar",
    plot_config=[
        {
            "x": df["Name"].values,
            "y": df["Number of olympics appearances"].values,
        }],
    layout_config=dict(
        title = "Zawodnicy z największa ilościa obecności na igrzyskach",
        xaxis_title = "Imie zawodnika",
        yaxis_title = "Ilość odbytych igrzysk",
        height = 900,
        width = 900
    )
)

In [131]:
build_plot(config)

INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Zawodnicy z największa ilością medali

In [132]:
query = """
WITH grouped_query as (SELECT Name,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Name)
SELECT * FROM grouped_query order by count desc limit 10;
"""

In [133]:
df = connection.query(query,
    output_columns=["Name","Number of medals"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Name,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Name)
SELECT * FROM grouped_query order by count desc limit 10;

INFO:root:Query executed successfully. The number of returned rows is 10


In [134]:
config = BaseConfig(
    plot_type="pie",
    plot_config=[
        {
            "labels": df["Name"].values,
            "values": df["Number of medals"].values,
            "visible":True 
        }],
    layout_config=dict(
        title = "Zawodnicy z największa ilościa medali",
        xaxis_title = "Imie zawodnika",
        yaxis_title = "Ilość zdobytych medali",
        legend_title = "Imie zawodnika",
        height = 1200,
        width = 1350
    )
)

In [135]:
build_plot(config)

INFO:root:Building the PiePlotter based on pie
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Zawodnicy z największa ilościa medali wobec danego państwa

In [136]:
query = """
WITH grouped_query as (SELECT Name,b.region,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null 
                       group by Name,b.region),
ranked_query as (SELECT Name,region,count,RANK() 
                      over (Partition BY region ORDER BY count DESC) 
                      FROM grouped_query)
SELECT * FROM ranked_query where rank <= 10;
"""

In [137]:
df = connection.query(query,
    output_columns=["Name","Country", "Number of medals","Rank"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Name,b.region,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null 
                       group by Name,b.region),
ranked_query as (SELECT Name,region,count,RANK() 
                      over (Partition BY region ORDER BY count DESC) 
                      FROM grouped_query)
SELECT * FROM ranked_query where rank <= 10;

INFO:root:Query executed successfully. The number of returned rows is 1862


In [138]:
config = BaseConfig(
    plot_type="bar",
    plot_config=[
        {
            "x": df[df["Country"] == country]["Name"].values,
            "y": df[df["Country"] == country]["Number of medals"].values,
            "name": str(country),
            "visible":True if i == 0 else "legendonly",
        }
        for i,country in enumerate([x for x in df["Country"].unique() if x is not None])
    ],
    layout_config=dict(
        title = "Zawodnicy z największa ilościa medali wobec danego państwa",
        xaxis_title = "Imie zawodnika",
        yaxis_title = "Ilość zdobytych medali",
        legend_title = "Państwo (Aby zmienić dyscypline należy dwukrotnie kliknąć pożądaną dyscypline.)",
        height = 1200,
        width = 1350
    )
)

In [139]:
build_plot(config)

INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Zawodnicy z największa ilościa medali wobec danej dyscypliny

In [140]:
query = """
WITH grouped_query as (SELECT Name,Sport,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Name,Sport),
ranked_query as (SELECT Name,Sport,count,RANK() 
                      over (Partition BY Sport ORDER BY count DESC) 
                      FROM grouped_query)
SELECT * FROM ranked_query where rank <= 10;
"""

In [141]:
df = connection.query(query,
    output_columns=["Name","Sport", "Number of medals","Rank"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Name,Sport,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Name,Sport),
ranked_query as (SELECT Name,Sport,count,RANK() 
                      over (Partition BY Sport ORDER BY count DESC) 
                      FROM grouped_query)
SELECT * FROM ranked_query where rank <= 10;

INFO:root:Query executed successfully. The number of returned rows is 1795


In [142]:
config = BaseConfig(
    plot_type="bar",
    plot_config=[
        {
            "x": df[df["Sport"] == sport]["Name"].values,
            "y": df[df["Sport"] == sport]["Number of medals"].values,
            "name": str(sport),
            "visible":True if i == 0 else "legendonly",
        }
        for i,sport in enumerate([x for x in df["Sport"].unique() if x is not None])
    ],
    layout_config=dict(
        title = "Zawodnicy z największa ilościa medali wobec danej dyscypliny",
        xaxis_title = "Imie zawodnika",
        yaxis_title = "Ilość zdobytych medali",
        legend_title = "Sport (Aby zmienić dyscypline należy dwukrotnie kliknąć pożądaną dyscypline.)",
        height = 1200,
        width = 1350
    )
)

In [143]:
build_plot(config)

INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Zawodnicy z największa ilościa poszczególnych medali

In [144]:
query = """
WITH grouped_query as (SELECT Name,Medal,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Name,Medal),
ranked_query as (SELECT Name,Medal,count,RANK() 
                      over (Partition BY Medal ORDER BY count DESC) 
                      FROM grouped_query)
SELECT * FROM ranked_query where rank <= 10;
"""

In [145]:
df = connection.query(query,
    output_columns=["Name","Medal", "Number of medals","Rank"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Name,Medal,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Name,Medal),
ranked_query as (SELECT Name,Medal,count,RANK() 
                      over (Partition BY Medal ORDER BY count DESC) 
                      FROM grouped_query)
SELECT * FROM ranked_query where rank <= 10;

INFO:root:Query executed successfully. The number of returned rows is 43


In [146]:
config = BaseConfig(
    plot_type="bar",
    plot_config=[
        {
            "x": df[df["Medal"] == medal]["Name"].values,
            "y": df[df["Medal"] == medal]["Number of medals"].values,
            "name": str(medal),
            "visible":True if i == 0 else "legendonly",
        }
        for i,medal in enumerate([x for x in df["Medal"].unique() if x is not None])
    ],
    layout_config=dict(
        title = "Zawodnicy z największa ilościa poszczególnych medali",
        xaxis_title = "Imie zawodnika",
        yaxis_title = "Ilość zdobytych medali",
        legend_title = "Medal (Aby zmienić rodzaj medalu należy dwukrotnie kliknąć pożądany medal.)",
        height = 1200,
        width = 1350,
        barmode='relative'
    )
)

In [147]:
build_plot(config)

INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Zawodnicy z największa ilościa medali w danym roku

In [148]:
query = """
WITH grouped_query as (SELECT Year,Name,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Year,Name),


ranked_query as (SELECT Year,Name,count,RANK() 
                      over (Partition BY Year ORDER BY count DESC) 
                      FROM grouped_query)

                      
SELECT * FROM ranked_query where rank <= 5
"""

In [149]:
df = connection.query(query,
    output_columns=["Year","Name", "Number of medals","Rank"],
)


INFO:root:Executing a query: 
WITH grouped_query as (SELECT Year,Name,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Year,Name),


ranked_query as (SELECT Year,Name,count,RANK() 
                      over (Partition BY Year ORDER BY count DESC) 
                      FROM grouped_query)

                      
SELECT * FROM ranked_query where rank <= 5

INFO:root:Query executed successfully. The number of returned rows is 331


In [150]:
config = BaseConfig(
    plot_type="bar",
    plot_config=[
        {
            "x": df[df["Year"] == year]["Name"].values,
            "y": df[df["Year"] == year]["Number of medals"].values,
            "name": str(year),
            "visible":True if i == 0 else "legendonly",
        }
        for i,year in enumerate(df["Year"].unique())
    ],
    layout_config=dict(
        title = "Zawodnicy z największa ilościa medali w danym roku",
        xaxis_title = "Imie zawodnika",
        yaxis_title = "Ilość zdobytych medali",
        legend_title = "Rok (Aby zmienić rok należy dwukrotnie kliknąć pożądaną datę.)",
        height = 1200,
        width = 1350
    )
)


In [151]:
build_plot(config)


INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Odsetek zawodników w zależności od płci na przestrzeni lat

In [152]:
query = """
WITH summed_up as (select distinct(Year),count(*) from athlete_events where Medal is not null group by Year),grouped_up as (SELECT distinct(Year),CASE WHEN Sex = 'M' THEN 'Male' When Sex = 'F' then 'Female' END AS Sex,count(*)\
                         FROM athlete_events\
                         where Medal is not null\
                         group by Year,Sex)\
    Select g.Year,g.Sex,round(g.count::decimal/s.count,2) from grouped_up as g join summed_up as s on g.Year = s.Year;"""

In [153]:
df = connection.query(
    query,
    output_columns=["Year", "Sex", "Percentage of athletes"],
)


INFO:root:Executing a query: 
WITH summed_up as (select distinct(Year),count(*) from athlete_events where Medal is not null group by Year),grouped_up as (SELECT distinct(Year),CASE WHEN Sex = 'M' THEN 'Male' When Sex = 'F' then 'Female' END AS Sex,count(*)                         FROM athlete_events                         where Medal is not null                         group by Year,Sex)    Select g.Year,g.Sex,round(g.count::decimal/s.count,2) from grouped_up as g join summed_up as s on g.Year = s.Year;
INFO:root:Query executed successfully. The number of returned rows is 69


In [154]:
config = BaseConfig(
    plot_type="bar",
    plot_config=[
        {
            "x": df[df["Sex"] == sex]["Year"].values,
            "y": df[df["Sex"] == sex]["Percentage of athletes"].values,
            "name": str(sex),
        }
        for sex in df["Sex"].unique()
    ],
    layout_config=dict(
        title = "Procent osób biorących udział w igrzyskach olimpijskich biorąc pod uwagę płeć na przestzeni lat",
        xaxis_title = "Rok",
        yaxis_title = "Odsetek danej płci",
        legend_title = "Płeć",
        height = 1200,
        width = 1350,
        barmode = "stack"
    )
)


In [155]:
build_plot(config)

INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Stosunek liczby zdobytych medali wobec danej płci, dyscypliny i stopnia medalu

In [156]:
query = """
WITH grouped_query as (SELECT Sport,Sex,Medal,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Sport,Sex,Medal)
SELECT * FROM grouped_query
"""

In [157]:
df = connection.query(
    query,
    output_columns=["Sport", "Sex","Medal","Number of medals"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Sport,Sex,Medal,count(*) 
                       from athlete_events 
                       where Medal is not null 
                       group by Sport,Sex,Medal)
SELECT * FROM grouped_query

INFO:root:Query executed successfully. The number of returned rows is 331


In [158]:
config = SubPlotsConfig(
    plot_type="subplot",
    plot_config=dict(
        rows=2,
        cols=1,
        specs=[[{"type": "bar"}],[{"type": "bar"}]],
    ),
    subplots_config=[
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Sport"] == sport]["Medal"].values,
                    "y": df[df["Sport"] == sport]["Number of medals"].values,
                    "name":sport, "legendgroup":sport,
                    "visible":True if i == 0 else "legendonly",
                }
                for i,sport in enumerate(df["Sport"].unique())
            ],
            layout_config = dict(
        barmode="group")
        ),
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Sport"] == sport]["Sex"].values,
                    "y": df[df["Sport"] == sport]["Number of medals"].values,
                    "name":sport, "legendgroup":sport,
                    "visible":True if i == 0 else "legendonly",
                }
                for i,sport in enumerate(df["Sport"].unique())
            ],
            layout_config = dict(
        barmode="group")
        ),
    ],
    layout_config=dict(
        title="Stosunek liczby zdobytych medali wobec danej płci, dyscypliny i stopnia medalu",
        xaxis_title="Dyscyplina",
        yaxis_title="Ilość wygranych medali",
        legend_title="Płeć & rodzaj medalu",
        height=1200,
        width=1350,
        barmode="stack",
    ),
)


In [159]:
build_plot(config)

INFO:root:Building the SubPlotter based on subplot with given plots: bar,bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


INFO:root:Plot rendered successfully!


## Ilość poszczególnych medali wobec danego państwa na przestzeni lat


In [160]:
query = """
WITH grouped_query as (SELECT Year,Medal,b.region,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null 
                       group by Year,b.region,Medal)
SELECT * FROM grouped_query ORDER BY Year,region,Medal;
"""

In [161]:
df = connection.query(
    query,
    output_columns=["Year", "Medal","Country","Number of medals"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Year,Medal,b.region,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null 
                       group by Year,b.region,Medal)
SELECT * FROM grouped_query ORDER BY Year,region,Medal;

INFO:root:Query executed successfully. The number of returned rows is 3219


In [162]:
config = SubPlotsConfig(
    plot_type="subplot",
    plot_config=dict(
        rows=2,
        cols=1,
        specs=[[{"type": "xy"}], [{"type": "bar"}]],
    ),
    subplots_config=[
        BaseConfig(
            plot_type="line",
            plot_config=[
                {
                    "x": df[df["Country"] == country]["Year"].values,
                    "y": df[df["Country"] == country]["Number of medals"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Country"] == country]["Medal"].values,
                    "y": df[df["Country"] == country]["Number of medals"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
    ],
    layout_config=dict(
        title="Ilość poszczególnych medali wobec danego państwa na przestzeni lat",
        xaxis_title="Rok/Rodzaj medalu",
        yaxis_title="Ilość wygranych medali",
        legend_title="Państwo",
        height=1200,
        width=1350,
        barmode="group",
    ),
)


In [163]:
build_plot(config)

INFO:root:Building the SubPlotter based on subplot with given plots: line,bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..
INFO:root:Building the LinePlotter based on line
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..
INFO:root:Plot rendered successfully!
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


INFO:root:Plot rendered successfully!


## Ilość zdobytych medali według płci, roku oraz poszczególnego państwa

In [164]:
query = """
WITH grouped_query as (SELECT Year,b.region,Sex,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null 
                       group by Year,b.region,Sex)
SELECT * FROM grouped_query ORDER BY Year,region,Sex;
"""

In [165]:
df = connection.query(
    query,
    output_columns=["Year","Country","Sex","Number of medals"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Year,b.region,Sex,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null 
                       group by Year,b.region,Sex)
SELECT * FROM grouped_query ORDER BY Year,region,Sex;

INFO:root:Query executed successfully. The number of returned rows is 2188


In [166]:
config = SubPlotsConfig(
    plot_type="subplot",
    plot_config=dict(
        rows=2,
        cols=1,
        specs=[[{"type": "bar"}], [{"type": "bar"}]],
    ),
    subplots_config=[
        BaseConfig(
            plot_type="pie",
            plot_config=[
                {
                    "labels": df[df["Country"] == country]["Year"].values,
                    "values": df[df["Country"] == country]["Number of medals"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Country"] == country]["Sex"].values,
                    "y": df[df["Country"] == country]["Number of medals"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
    ],
    layout_config=dict(
        title="Ilość poszczególnych medali wobec danego państwa na przestzeni lat",
        xaxis_title="Rok/Rodzaj medalu",
        yaxis_title="Ilość wygranych medali",
        legend_title="Państwo",
        height=1200,
        width=1350,
        barmode="group",
    ),
)

In [167]:
build_plot(config)

INFO:root:Building the SubPlotter based on subplot with given plots: pie,bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..
INFO:root:Building the PiePlotter based on pie
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


TypeError: _add_traces() got an unexpected keyword argument 'subplot'

## Ilość wystąpień danego państwa wobec roku

In [168]:
query = """
WITH grouped_query as (SELECT b.region,Year,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       group by b.region,Year)
SELECT * FROM grouped_query ORDER BY Year,region;
"""

In [169]:
df = connection.query(
    query,
    output_columns=["Year","Country","Number of appearances"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT b.region,Year,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       group by b.region,Year)
SELECT * FROM grouped_query ORDER BY Year,region;

INFO:root:Query executed successfully. The number of returned rows is 3278


In [170]:
config = BaseConfig(
    plot_type="bar",
    plot_config=[
        {
            "x": df[df["Year"] == year]["Country"].values,
            "y": df[df["Year"] == year]["Number of appearances"].values,
            "name": str(year),
            "visible": True if i == 0 else "legendonly",
        }
        for i,year in enumerate(df["Year"].unique())
    ],
    layout_config=dict(
        title = "Ilość wystąpień w igrzyskach olimpijskich na przestzeni lat poszczególnych państw.",
        xaxis_title = "Rok",
        yaxis_title = "Ilość wystąpień",
        legend_title = "Państwo",
        height = 1200,
        width = 1350,
        barmode = "group"
    )
)

In [171]:
build_plot(config)

INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


## Ilość dyscyplin wobec roku

In [172]:
query = """
WITH grouped_query as (SELECT Sport,Year,count(*) 
                       from athlete_events
                       group by Sport,Year)
SELECT * FROM grouped_query ORDER BY Year,Sport;
"""

In [173]:
df = connection.query(
    query,
    output_columns=["Year","Sport","Number of events"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT Sport,Year,count(*) 
                       from athlete_events
                       group by Sport,Year)
SELECT * FROM grouped_query ORDER BY Year,Sport;

INFO:root:Query executed successfully. The number of returned rows is 906


## Ilość wystąpień danego państwa wobec danej dyscypliny oraz roku

In [174]:
query = """
WITH grouped_query as (SELECT b.region,Sport,Year,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       group by b.region,Sport,Year)
SELECT * FROM grouped_query ORDER BY Year,region,Sport;
"""

In [175]:
df = connection.query(
    query,
    output_columns=["Country","Sport","Year","Number of appearances"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT b.region,Sport,Year,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       group by b.region,Sport,Year)
SELECT * FROM grouped_query ORDER BY Year,region,Sport;

INFO:root:Query executed successfully. The number of returned rows is 26560


In [176]:
df.head()

,Country,Sport,Year,Number of appearances
0,Australia,Athletics,1896,3
1,Australia,Tennis,1896,2
2,Austria,Cycling,1896,4
3,Austria,Fencing,1896,1
4,Austria,Swimming,1896,3


In [177]:
config = SubPlotsConfig(
    plot_type="subplot",
    plot_config=dict(
        rows=2,
        cols=1,
        specs=[[{"type": "bar"}], [{"type": "bar"}]],
    ),
    subplots_config=[
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Country"] == country]["Year"].values,
                    "y": df[df["Country"] == country]["Number of appearances"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Country"] == country]["Sport"].values,
                    "y": df[df["Country"] == country]["Number of appearances"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
    ],
    layout_config=dict(
        title="Ilość poszczególnych wystąpień danego państwa wobec sportu na przestzeni lat",
        xaxis_title="Rok/Sport",
        yaxis_title="Ilość wystąpień",
        legend_title="Państwo",
        height=1200,
        width=1350,
        barmode="group",
    ),
)

In [178]:
build_plot(config)

INFO:root:Building the SubPlotter based on subplot with given plots: bar,bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


INFO:root:Plot rendered successfully!


## Ilość medali danego państwa wobec danej dyscypliny oraz roku

In [179]:
query = """
WITH grouped_query as (SELECT b.region,Sport,Year,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null
                       group by b.region,Sport,Year)
SELECT * FROM grouped_query ORDER BY Year,region,Sport;
"""

In [180]:
df = connection.query(
    query,
    output_columns=["Country","Sport","Year","Number of medals"],
)

INFO:root:Executing a query: 
WITH grouped_query as (SELECT b.region,Sport,Year,count(*) 
                       from athlete_events a
                       join noc_regions b
                       on b.NOC = a.NOC
                       where Medal is not null
                       group by b.region,Sport,Year)
SELECT * FROM grouped_query ORDER BY Year,region,Sport;

INFO:root:Query executed successfully. The number of returned rows is 6863


In [181]:
config = SubPlotsConfig(
    plot_type="subplot",
    plot_config=dict(
        rows=2,
        cols=1,
        specs=[[{"type": "bar"}], [{"type": "bar"}]],
    ),
    subplots_config=[
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Country"] == country]["Sport"].values,
                    "y": df[df["Country"] == country]["Number of medals"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
        BaseConfig(
            plot_type="bar",
            plot_config=[
                {
                    "x": df[df["Country"] == country]["Year"].values,
                    "y": df[df["Country"] == country]["Number of medals"].values,
                    "name": country,
                    "legendgroup": country,
                    "visible": True if i == 0 else "legendonly",
                    "showlegend": False
                }
                for i, country in enumerate(df["Country"].unique())
            ],
            layout_config=dict(),
        ),
    ],
    layout_config=dict(
        title="Ilość poszczególnych medali danego państwa wobec sportu na przestzeni lat",
        xaxis_title="Rok/Sport",
        yaxis_title="Ilość medali",
        legend_title="Państwo",
        height=1200,
        width=1350,
        barmode="group",
    ),
)
build_plot(config)

INFO:root:Building the SubPlotter based on subplot with given plots: bar,bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!
INFO:root:Building the BarPlotter based on bar
INFO:root:Configuring the plot parameters..
INFO:root:Displaying the plot..


INFO:root:Plot rendered successfully!


INFO:root:Plot rendered successfully!
